In [1]:
from templates.templates import Template, get_template
from transformers import AutoTokenizer

In [2]:
template = Template(
    name="qwen",
    system_template="<|im_start|>system\n{system_message}<|im_end|>\n",
    system_message="You are a helpful assistant.",
    user_template="<|im_start|>user\n{content}<|im_end|>\n",
    assistant_template="<|im_start|>assistant\n{content}<|im_end|>",
    tool_template="<|im_start|>user\n<tool_response>\n{observation}\n</tool_response>"
)

In [2]:
template = get_template("qwen2.5")

In [3]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Hello, how are you today?",
            }
        ]
    },
    {
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": "Hi, I am good."
            }
        ]
    }
]

In [4]:
prompt, _, _ = template.render(messages, add_generation_prompt=True)

In [5]:
print(prompt)

<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Hello, how are you today?<|im_end|>
<|im_start|>assistant
Hi, I am good.<|im_end|>
<|im_start|>assistant



In [6]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct")

In [7]:
tokenizer.chat_template = template.jinja_template()
print(tokenizer.chat_template)

{% set _u_pref  = '<|im_start|>user\n' %}{% set _u_suff  = '<|im_end|>\n' %}{% set _a_pref  = '<|im_start|>assistant\n' %}{% set _a_suff  = '<|im_end|>\n' %}{% set _t_pref  = '<|im_start|>tool\n' %}{% set _t_suff  = '<|im_end|>\n' %}{% set _img_tok = '<|vision_start|><|image_pad|><|vision_end|>' %}{% set _vid_tok = '<|vision_start|><|video_pad|><|vision_end|>' %}{% if messages and messages[0]['role'] == 'system' %}{{ messages[0]['content'] }}{% else %}<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
{% endif %}{% for m in messages %}{% if m['role'] == 'user' %}{% set ns = namespace(txt='') %}{% for item in m['content'] %}{% if item['type'] == 'text'  %}{% set ns.txt = ns.txt + item['text'] %}{% elif item['type'] == 'image' %}{% set ns.txt = ns.txt + _img_tok %}{% elif item['type'] == 'video' %}{% set ns.txt = ns.txt + _vid_tok %}{% endif %}{% endfor %}{{ _u_pref }}{{ ns.txt }}{{ _u_suff }}{% elif m['role'] == 'assistant' %}{{ _a_pref }}{

In [10]:
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(prompt)

<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Hello, how are you today?<|im_end|>
<|im_start|>assistant
Hi, I am good.<|im_end|>
<|im_start|>assistant



In [5]:
inputs = template.encode(messages, tokenizer, return_tensors="pt")

NameError: name 'tokenizer' is not defined

In [14]:
print(inputs)

{'input_ids': tensor([[151644,   8948,    198,   2610,    525,   1207,  16948,     11,   3465,
            553,  54364,  14817,     13,   1446,    525,    264,  10950,  17847,
             13, 151645,    198, 151644,  77091,    198,  13048,     11,    358,
           1079,   1661,     13, 151645,    198]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([[  -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,  13048,     11,    358,
           1079,   1661,     13, 151645,   -100]]), 'action_mask': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         1, 1, 1, 1, 1, 1, 1, 0]])}


In [15]:
tokenizer.decode([151644, 8948, 198, 2610, 525, 1207, 16948, 11, 3465, 553, 54364, 14817, 13, 1446, 525, 264, 10950, 17847, 13, 151645, 198, 151644, 77091, 198, 13048, 11, 358, 1079, 1661, 13, 151645, 198])

'<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>assistant\nHi, I am good.<|im_end|>\n'

In [16]:
tokenizer.decode([198])

'\n'

In [3]:
messages = [
    {"role": "system", "content": [{"type": "text", "text": "You are a helpful assistant."}]},
    {"role": "user", "content": [{"type": "text", "text": "Hello, how are you?"}]},
    {"role": "assistant", "content": [{"type": "text", "text": "I am fine, thank you."}]},
    {"role": "user", "content": [{"type": "text", "text": "What is 3 times 5?"}]},
]

In [4]:
prompt, elements, roles = template.render(messages)
elements, mask_flags = template._postprocess_elements(elements, roles)

In [5]:
print(elements)
print(mask_flags)

['<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nHello, how are you?<|im_end|>\n<|im_start|>assistant\n', 'I am fine, thank you.<|im_end|>', '\n<|im_start|>user\nWhat is 3 times 5?<|im_end|>\n']
[True, False, True]
